In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero
from torch_geometric.data import HeteroData
from torch_geometric.loader import DataLoader
from torch import nn
from torch_geometric.utils import from_dgl
from tqdm import tqdm

In [2]:
import data.fraud_dataset as fraud_dataset

DATASET_NAME = "yelp"
TRAIN_SIZE = 0.4
VAL_SIZE = 0.1
RANDOM_SEED = 42
FORCE_RELOAD = False

In [3]:
fraud_data = fraud_dataset.FraudDataset(
    DATASET_NAME, 
    train_size=TRAIN_SIZE, 
    val_size=VAL_SIZE, 
    random_seed=RANDOM_SEED, 
    force_reload=FORCE_RELOAD
)
graph = fraud_data[0]

data = from_dgl(graph)
data.metadata

Done loading data from cached files.


<bound method HeteroData.metadata of HeteroData(
  review={
    test_mask=[45954],
    val_mask=[45954],
    train_mask=[45954],
    label=[45954],
    feature=[45954, 32],
  },
  (review, net_rsr, review)={ edge_index=[2, 6805486] },
  (review, net_rtr, review)={ edge_index=[2, 1147232] },
  (review, net_rur, review)={ edge_index=[2, 98630] }
)>

In [4]:
class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, dropout=0.6):
        super(GAT, self).__init__()
        # First GAT convolution layer
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, concat=True, dropout=dropout, bias=False)
        # Second GAT convolution layer
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=True, bias=False)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.conv1(x, edge_index))  # Apply the first GATConv layer
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)  # Apply the second GATConv layer
        return x

In [5]:
# from torch_geometric.nn.models import GAT

# Creating a model instance covering heterogeneity
# model = GAT(in_channels=32 ,hidden_channels=32, num_layers=2, out_channels=2)

model = GAT(in_channels=32 ,hidden_channels=32, heads=2, out_channels=2)
model = to_hetero(model, data.metadata(), aggr='sum', debug=True)

opcode         name        target                                args                     kwargs
-------------  ----------  ------------------------------------  -----------------------  ----------------------------------------------
placeholder    x           x                                     ()                       {}
placeholder    edge_index  edge_index                            ()                       {}
call_function  dropout     <function dropout at 0x7fbf581e7d80>  (x,)                     {'p': 0.6, 'training': True, 'inplace': False}
call_module    conv1       conv1                                 (dropout, edge_index)    {}
call_function  elu         <function elu at 0x7fbf57ff44a0>      (conv1,)                 {'alpha': 1.0, 'inplace': False}
call_function  dropout_1   <function dropout at 0x7fbf581e7d80>  (elu,)                   {'p': 0.6, 'training': True, 'inplace': False}
call_module    conv2       conv2                                 (dropout_1, edge_index)  

/home/kaifeng/Documents/gnn_project/LAGAT/LAGAT/lib/python3.12/site-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "
/home/kaifeng/Documents/gnn_project/LAGAT/LAGAT/lib/python3.12/site-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout_1' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
data = data.to(device)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.feature_dict, data.edge_index_dict)

    train_mask = data['review'].train_mask.to(device)
    label = data['review'].label.to(device)

    logits = out['review'][train_mask.bool()]
    targets = label[train_mask.bool()].long()

    loss = F.cross_entropy(logits, targets)

    loss.backward()
    optimizer.step()
    return loss.item()

n_epochs = 300
progress_bar = tqdm(range(n_epochs), desc='Training')

for epoch in progress_bar:
    loss = train()
    progress_bar.set_postfix({'Loss': f'{loss:.4f}'})

Training: 100%|██████████| 300/300 [00:30<00:00,  9.79it/s, Loss=0.3844]


In [9]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score


def test(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.feature_dict, data.edge_index_dict)
        scores = torch.softmax(out['review'], dim=1)  # Convert logits to probabilities

    labels = data['review'].label.cpu()
    pred = scores.argmax(dim=1).cpu()

    def calc_metrics(target_mask):
        mask_indices = target_mask.cpu()
        masked_labels = labels[mask_indices.bool()]
        masked_pred = pred[mask_indices.bool()]
        masked_scores = scores[mask_indices.bool()][:, 1].cpu()

        f1 = f1_score(masked_labels, masked_pred, average='macro')
        try:
            auc = roc_auc_score(masked_labels, masked_scores)
            ap = average_precision_score(masked_labels, masked_scores)
        except Exception as e:
            print(e)
            auc, ap = float('nan'), float('nan')  # In case of an exception (like only one class present), return NaN
        return f1, auc, ap

    train_metrics = calc_metrics(data['review'].train_mask)
    val_metrics = calc_metrics(data['review'].val_mask)
    test_metrics = calc_metrics(data['review'].test_mask)

    print('--- Training Metrics ---')
    print(f'F1 Score: {train_metrics[0]:.4f}, AUC: {train_metrics[1]:.4f}, AP: {train_metrics[2]:.4f}')
    
    print('--- Validation Metrics ---')
    print(f'F1 Score: {val_metrics[0]:.4f}, AUC: {val_metrics[1]:.4f}, AP: {val_metrics[2]:.4f}')
    
    print('--- Test Metrics ---')
    print(f'F1 Score: {test_metrics[0]:.4f}, AUC: {test_metrics[1]:.4f}, AP: {test_metrics[2]:.4f}')

    return {'train': train_metrics, 'val': val_metrics, 'test': test_metrics}

_ = test(model, data)

--- Training Metrics ---
F1 Score: 0.4617, AUC: 0.7097, AP: 0.2896
--- Validation Metrics ---
F1 Score: 0.4629, AUC: 0.7028, AP: 0.2540
--- Test Metrics ---
F1 Score: 0.4621, AUC: 0.6992, AP: 0.2751


In [8]:
# There is multi-step label leakage in training, might not learn anything, might learn something
# This way can avoid using ego network (which cannot be run efficiently)